In [1]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

In [7]:
import pandas as pd
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer
from geopy.distance import geodesic


In [5]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    

In [6]:
# I USED THIS FOR DEBUGGING
embeddings = model.encode('900 Embarcadero Del Mar')
print(len(embeddings))
output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value='Hotels',
                    ),
                )
            ]
        ),
        query_vector=embeddings,
        limit=3,
    )
print(output)

384
[ScoredPoint(id='5e744e0c-cee7-5cf6-b25f-bf4bbf6a6dd2', version=61, score=0.5553415, payload={'categories': ['Hotels', 'Hotels & Travel', 'Event Planning & Services'], 'full_address': '424 Por La Mar Santa Barbara 93103', 'hours': None, 'latitude': 34.4189383557, 'longitude': -119.6712183952, 'name': 'Parkside Inn', 'review_count': 42, 'stars': 3.0}, vector=None, shard_key=None), ScoredPoint(id='c557b509-606f-5a9e-9435-1bdc0d65cc07', version=98, score=0.53008175, payload={'categories': ['Venues & Event Spaces', 'Hotels & Travel', 'Event Planning & Services', 'Hotels', 'Wedding Planning'], 'full_address': '900 San Ysidro Ln Santa Barbara 93108', 'hours': {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0', 'Thursday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'Wednesday': '0:0-0:0'}, 'latitude': 34.4468773902, 'longitude': -119.623055, 'name': 'San Ysidro Ranch', 'review_count': 129, 'stars': 4.5}, vector=None, shard_key=None), ScoredPoint(id='5faa0e46-12ef-

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [18]:
class SearchInput(BaseModel):
    location: str = Field(description="Current location of user.")
    Category: str = Field(description="Type of business that the user wants to go.")
@tool("get_category", args_schema=SearchInput)
def get_category(location: str, Category: str) -> str:
    """Get a business recommendation given location and category of user's choice.\
        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea """
    print(location, Category)
    address_embedding = model.encode(location)
    # print(address_embedding)
    output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value=Category,
                    ),
                )
            ]
        ),
        
        query_vector=address_embedding,
        limit=3,
    )

    print('Query output:',output)
    best_point = max(output, key=lambda x: x.score)
    poi_name = best_point.payload['name']
    poi_address = best_point.payload['full_address']

    print(f"The best-rated {Category} is {poi_name} located at {poi_name}.")

    return {"Name": poi_name ,"Location": poi_address}

tools = [get_category]
functions = [convert_to_openai_function(t) for t in tools]
functions

[{'name': 'get_category',
  'description': 'get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Current location of user.',
     'type': 'string'},
    'Category': {'description': 'Type of business that the user wants to go.',
     'type': 'string'}},
   'required': ['location', 'Category']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [19]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are an navigation assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are an navigation assistant with access to these functions -
[
    {
        "name": "get_category",
        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user's choice.        e.g. \"Name of restaurant\": Helena Avenue Bakery, \"Location\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\"Category\": Food, Restaurants, Salad, Coffee & Tea",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "Current location of user.",
                    "type": "string"
                },
                "Category": {
                    "description": "Type of business that the user wants to go.",
                    "type": "string"
                }
            },
            "required": [
                "location",
                "Category"
            ]
        }
    }
]



In [20]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    print(match)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)
            arguments = json.loads(arguments_str)
            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            return None
    return None


In [143]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'Im at 900 Embarcadero Del Mar, any cafes nearby?'},
]

response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
message = (response['message']['content'])

In [144]:
message

'<functioncall> {"name": "get_category", "arguments": \'{"location": "900 Embarcadero Del Mar", "Category": "Cafes"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [145]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json
print(function_call)
if function_call and function_call.get("name") == "get_category":
    args = function_call.get("arguments")
    restaurant = get_category.run(args)

restaurant

<re.Match object; span=(16, 115), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '900 Embarcadero Del Mar', 'Category': 'Cafes'}}
900 Embarcadero Del Mar Cafes
Query output: [ScoredPoint(id='ef208299-9d31-5b57-aee8-5ea21f57c4a8', version=23, score=0.5160066, payload={'categories': ['Cafes', 'Restaurants', 'Breakfast & Brunch'], 'full_address': '3007 De La Vina St Santa Barbara 93105', 'hours': {'Friday': '7:0-14:0', 'Monday': '7:0-14:0', 'Saturday': '7:0-14:0', 'Sunday': '7:0-13:0', 'Thursday': '7:0-14:0', 'Tuesday': '7:0-14:0', 'Wednesday': '7:0-14:0'}, 'name': 'The Patio Cafe Santa Barbara', 'review_count': 65, 'stars': 3.0}, vector=None, shard_key=None), ScoredPoint(id='519d47f5-851e-51a5-9d90-200d2abc85ee', version=0, score=0.50789714, payload={'categories': ['Restaurants', 'Cafes', 'Breakfast & Brunch', 'Bakeries', 'Food'], 'full_address': '2700 De La Vina St Santa Barbara 93105', 'hours': {'Friday': '6:30-15:30', 'Mond

{'Name': 'The Patio Cafe Santa Barbara',
 'Location': '3007 De La Vina St Santa Barbara 93105'}

In [33]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(restaurant)})
messages

[{'role': 'system',
  'content': '\nYou are an navigation assistant with access to these functions -\n[\n    {\n        "name": "get_category",\n        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. \\"Name of restaurant\\": Helena Avenue Bakery, \\"Location\\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\\"Category\\": Food, Restaurants, Salad, Coffee & Tea",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "Current location of user.",\n                    "type": "string"\n                },\n                "Category": {\n                    "description": "Type of business that the user wants to go.",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location",\n                "Category"\n            ]\n

In [34]:
response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
response['message']['content']

"I found a hotel for you. It's called Parkside Inn. It is located at 424 Por La Mar, Santa Barbara 93103"

# Evalulate the model

## Test truthfulness

In [61]:
def isTruthful(name, address):
    doesNameExist = client.search(
            collection_name=f"{collection_name}",
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="name",
                        match=models.MatchValue(
                            value=name,
                        ),
                    )
                ]
            ),
            query_vector=embeddings,
            limit=1,
        )
    doesAddressExist = client.search(
            collection_name=f"{collection_name}",
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="full_address",
                        match=models.MatchValue(
                            value=address,
                        ),
                    )
                ]
            ),
            query_vector=embeddings,
            limit=1,
        )

    if doesNameExist and doesAddressExist:
        print("YES")
        return True
    else:
        print("FK NO")
        return False

def isNear(minDist):
    

def generateLLMOutput(message):
    function_call = parse_function_call(message) # parse out function call name and args into json
    print(function_call)
    if function_call and function_call.get("name") == "get_category":
        args = function_call.get("arguments")
        output = get_category.run(args)
        return output
    return None


In [146]:
def evaluateModel():
    numTruthful = 0
    numTested = 0
    locations = [
        ["1600 Amphitheatre Parkway", 37.4220, -122.0841],
        ["900 Embarcadero Del Mar", 34.4127, -119.8548],
        ["2211 North First Street", 37.3753, -121.9229],
        ["555 California Street", 37.7910, -122.4028],
        ["1200 Getty Center Drive", 34.0790, -118.4751],
        ["100 Universal City Plaza", 34.1381, -118.3534],
        ["2000 Avenue of the Stars", 34.0592, -118.4179],
        ["500 Parnassus Avenue", 37.7631, -122.4580],
        ["800 N State College Blvd", 33.8798, -117.8870],
        ["1 Infinite Loop", 37.3318, -122.0312]
    ]
    categories = ['restaurants', 'hotels', 'churches', 'bars', 'cafes']

    for i in locations:
        for j in categories:
            try:
                response = ollama.chat(model='calebfahlgren/natural-functions', messages=[{'role': 'system','content': SYSTEM_PROMPT},
                                                                                        {'role': 'user','content': f"Im at {i[0]}, any {j} nearby?"}])
                message = (response['message']['content'])
                output = generateLLMOutput(message)            
                if (output and isTruthful(output['Name'], output['Location'])):
                    numTruthful += 1
            except Exception as e:
                pass
            else:
                numTested += 1
        
    truthfullnessScore = numTruthful / numTested * 100;
    print("Truthfullness Score: ", truthfullnessScore, "%")

evaluateModel()

<re.Match object; span=(16, 131), match='"name": "get_category", "arguments": \'{"location>
None
FK NO
<re.Match object; span=(16, 117), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '1600 Amphitheatre Parkway', 'Category': 'Hotel'}}
1600 Amphitheatre Parkway Hotel
Query output: []
<re.Match object; span=(16, 120), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '1600 Amphitheatre Parkway', 'Category': 'churches'}}
1600 Amphitheatre Parkway churches
Query output: []
<re.Match object; span=(16, 116), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '1600 Amphitheatre Parkway', 'Category': 'Bars'}}
1600 Amphitheatre Parkway Bars
Query output: [ScoredPoint(id='9f0f701b-0bfd-5a29-bea2-a36896919776', version=2, score=0.43017605, payload={'categories': ['Seafood', 'Venues & Event Spaces', 'Cocktail Bars', 'Rest